In [24]:
## this code use connection to a dataBase
## it conatin a GUI interface

## if you face any problem running this code try the last call below

In [ ]:
## the DataBase contain table named Qustions with "id","translated_qustion","question","answer"
## the translated qustion is found by translate(question) then stored in the DataBased

In [3]:
!pip install transformers

In [4]:
!pip install sentencepiece

In [41]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d3/1d/a257913c89572de61316461db91867f87519146e58132cdeace3d9ffbe1f/torch-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhosted.org/packages/fe/1c/5f6dbf18e8b73e0a5472466f0ea8d48ce9efae39bd2ff38cebf8dce61259/mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Obtaining dependency information for intel-openmp==2021.* from https://files.pythonhosted.org/packages/6f/21/b590c0cc3888b24

ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pyodbc
import tkinter as tk
from tkinter import messagebox
from tkinter import font as tkfont

In [ ]:
#load the translate model and tokenizer
tokenizer_translate = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model_translate = TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

In [ ]:
def translate(sentence):
  # Encode the text
  input_ids = tokenizer_translate.encode(sentence, return_tensors="tf")

  # Generate the translated text
  outputs = model_translate.generate(input_ids)
  translated_text = tokenizer_translate.decode(outputs[0], skip_special_tokens=True)

  return translated_text

In [ ]:
# Load bert pre-trained model and tokenizer
tokenizer_bert = AutoTokenizer.from_pretrained('bert-large-uncased')
model_bert = AutoModel.from_pretrained('bert-large-uncased')

In [ ]:
def bert_similarity(sentence1 , sentence2):

#   translated_sentence1 = translate(sentence1)
#   translated_sentence2 = translate(sentence2)
  translated_sentence1 = sentence1
  translated_sentence2 = sentence2

  # Encode sentences and compute embeddings
  inputs1 = tokenizer_bert(translated_sentence1, return_tensors='pt')
  inputs2 = tokenizer_bert(translated_sentence2, return_tensors='pt')

  with torch.no_grad():
      embeddings1 = model_bert(**inputs1).last_hidden_state.mean(dim=1)
      embeddings2 = model_bert(**inputs2).last_hidden_state.mean(dim=1)

  # Compute cosine similarity
  similarity_bert = cosine_similarity(embeddings1, embeddings2)

  return similarity_bert

In [4]:
def get_translated_qustions():
    #connect to the database
    server = 'server name' 
    database = 'database naem'
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
    
    cursor = cnxn.cursor()
    #load the english qustions
    cursor.execute('SELECT translated_question, id from Questions')

    pairs_question_id = []
    for row in cursor:
        pairs_question_id.append((row.translated_question, row.id))
        
    cnxn.close()
    
    return pairs_question_id

In [13]:
def get_similar_sentence(user_sentence):
    
    translated_user_sentence = translate(user_sentece)
    
    # Get the list of pairs from the get_translated_questions function
    translated_questions = get_translated_questions()

    similarity_scores = []
    for question, id in translated_questions:
        # Calculate the similarity score between the user sentence and the current question
        similarity = bert_similarity(translated_user_sentence, question)
        similarity_scores.append((similarity, question, id))

    # Sort the list in descending order of similarity score
    similarity_scores.sort(key=lambda x: x[0], reverse=True)

    # Return the top 3 most similar questions
    return similarity_scores[:3]

In [23]:
server = 'server name' 
database = 'database name'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

cursor = cnxn.cursor()

def submit():
    sentence = entry.get()
    results = get_similar_sentence(sentence)
    result_text.delete(1.0, tk.END)
    question_buttons.clear()
    for result in results:
        cursor.execute('SELECT question, answer FROM Questions WHERE id = ?', (result[2],))
        question, answer = cursor.fetchone()
        

        start_index = result_text.index(tk.END)
        result_text.insert(tk.END, f"Question: {question}\n", f"question_tag_{result[2]}")
        

        button_frame = tk.Frame(root, bg='lightblue')
        show_answer_button = tk.Button(button_frame, text="Show Answer", command=lambda q_id=result[2], btn_frame=button_frame: show_answer(q_id, btn_frame), bg='darkblue', fg='white', font=('Helvetica', 10, 'bold'))
        show_answer_button.pack(side=tk.LEFT, padx=5)
        

        result_text.window_create(tk.END, window=button_frame)
        result_text.insert(tk.END, "\n\n") 
        

        question_buttons[result[2]] = False

def show_answer(q_id, button_frame):
    if question_buttons[q_id]:
        return
    
    cursor.execute('SELECT question, answer FROM Questions WHERE id = ?', (q_id,))
    question, answer = cursor.fetchone()
    

    question_index = result_text.search(f"Question: {question}", "1.0", tk.END)
    if question_index:
        result_text.insert(f"{question_index} lineend", f"\nAnswer: {answer}\n")
        question_buttons[q_id] = True
    

    button_frame.destroy()

root = tk.Tk()
root.geometry('400x500')
root.configure(bg='lightblue')

# Set a nice font for the labels and text
title_font = tkfont.Font(family='Helvetica', size=16, weight='bold')
text_font = tkfont.Font(family='Helvetica', size=14)
button_font = tkfont.Font(family='Helvetica', size=12, weight='bold')

label = tk.Label(root, text="Enter a sentence:", bg='lightblue', fg='darkblue', font=title_font, pady=10)
label.pack()

entry = tk.Entry(root, font=text_font, width=40, bd=2, relief=tk.GROOVE)
entry.pack(pady=5)

submit_button = tk.Button(root, text="Submit", command=submit, bg='darkblue', fg='white', font=button_font, pady=5)
submit_button.pack(pady=10)

result_text = tk.Text(root, bg='lightblue', fg='darkblue', font=text_font, wrap=tk.WORD, relief=tk.GROOVE, bd=2)
result_text.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

question_buttons = {}

root.mainloop()
cnxn.close()

In [ ]:
# # First, let's create the SQLite database from the CSV file
# import sqlite3
# import pandas as pd

# # Read the CSV file into a pandas DataFrame
# df = pd.read_csv('data.csv')

# # Connect to SQLite database (or create it if it doesn't exist)
# conn = sqlite3.connect('database.db')

# # Write the DataFrame to the SQLite database
# df.to_sql('Questions', conn, if_exists='replace', index=False)

# conn.close()

# # Now, let's define the functions to compute BERT similarity and fetch questions from the database
# from transformers import AutoTokenizer, AutoModel
# from sklearn.metrics.pairwise import cosine_similarity
# import torch

# # Load BERT pre-trained model and tokenizer
# tokenizer_bert = AutoTokenizer.from_pretrained('bert-large-uncased')
# model_bert = AutoModel.from_pretrained('bert-large-uncased')

# #load the translate model and tokenizer
# tokenizer_translate = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
# model_translate = TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

# def translate(sentence):
#     # Encode the text
#     input_ids = tokenizer_translate.encode(sentence, return_tensors="tf")

#     # Generate the translated text
#     outputs = model_translate.generate(input_ids)
#     translated_text = tokenizer_translate.decode(outputs[0], skip_special_tokens=True)

#     return translated_text

# def bert_similarity(sentence1, sentence2):
#     # Encode sentences and compute embeddings
#     inputs1 = tokenizer_bert(sentence1, return_tensors='pt')
#     inputs2 = tokenizer_bert(sentence2, return_tensors='pt')

#     with torch.no_grad():
#         embeddings1 = model_bert(**inputs1).last_hidden_state.mean(dim=1)
#         embeddings2 = model_bert(**inputs2).last_hidden_state.mean(dim=1)

#     # Compute cosine similarity
#     similarity_bert = cosine_similarity(embeddings1.numpy(), embeddings2.numpy())

#     return similarity_bert.item()

# def get_questions():
#     conn = sqlite3.connect('my_database.db')
#     cursor = conn.cursor()
#     cursor.execute('SELECT translated_question, id FROM Questions')
#     pairs_question_id = [(row[0], row[1]) for row in cursor.fetchall()]
#     conn.close()
#     return pairs_question_id

# def get_similar_sentence(user_sentence):
#     questions = get_questions()
#     similarity_scores = [(bert_similarity(user_sentence, question), question, id) for question, id in questions]
#     similarity_scores.sort(key=lambda x: x[0], reverse=True)
#     return similarity_scores[:3]

# def main():
#     user_sentence_arabic = input("Enter a sentence: ")
#     user_sentence = translate(user_sentence_arabic)
#     results = get_similar_sentence(user_sentence)

#     for result in results:
#         print(f"Question: {result[1]} (Similarity: {result[0]:.2f})")
#         conn = sqlite3.connect('my_database.db')
#         cursor = conn.cursor()
#         cursor.execute('SELECT question, answers FROM Questions WHERE id = ?', (result[2],))
#         row = cursor.fetchone()
#         question, answer = row[0], row[1]
#         print(f"Original Question: {question}")
#         print(f"Answer: {answer}\n")
#         conn.close()

# if __name__ == "__main__":
#     main()